**Streamlit**

Streamlit est une bibliothèque Python qui permet de créer des application web sans avoir besoin de savoir le développement web. 

On écrit les différentes parties dans l'ordre qu'on souhaite s'afficher sur la page web. 
- ``st.title(title:str)`` permet d'afficher un titre
- ``st.write(text:str)`` permet d'écrire un texte
- ``st.file_uploader()`` permet d'ajouter un bouton ``upload`` pour importer des fichiers. Le fichier importé sera sous la forme de bytes.

Streamlit permet aussi de mettre des données en cache. Cela est utile pour enregistrer les résultats de fonctions qu'on utilise souvent avec les mêmes paramètres (load_model par exemple). Donc, Streamlit va calculer la fonction seulement pour la première exécution. Cela est possible avec la synatxe ``@st.cache()`` qu'on écrit avant la fonction qu'on veut enregistrer.


In [ ]:
#install dependencies
!pip install streamlit
!pip install boto3

**Utils**

In [5]:
from PIL import Image
import torch
from model import ResNet
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import streamlit as st
import boto3
import pickle

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP']

#@st.cache()
def import_model(bucket, key="diffusion/state_dict.pickle", device=torch.device('cpu')):
    s3 = boto3.client('s3',endpoint_url='https://minio.lab.sspcloud.fr/')
    data = s3.get_object(Bucket=bucket, Key=key)
    state_dict = pickle.loads(data['Body'].read())
    model = ResNet()
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model

def load_image(file):
    img = Image.open(file).convert("RGB")
    return img

#@st.cache()
def predict(img, model, device):
    img = TF.to_tensor(img)
    img = TF.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    img = img.to(device)
    img = img.unsqueeze(0)
    preds = model(img)
    preds = F.softmax(preds, dim=1)
    if float(preds[0][0]) < float(preds[0][1]):
        results = "There is a ship!"
    else:
        results = "There is no ship!"
    return results

@st.cache()
def load_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    return device
@st.cache()
def is_image_file(filename):
  return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

**main**

In [6]:
import torch

device = load_device()
model = import_model(bucket="mbenxsalha", key="diffusion/state_dict.pickle", device=device)

st.title("Welcome to The Ship Detective!")
st.write("The image you upload will be fed to a Deep Neural Network in real-time to verify if there is a ship or not")
file = st.file_uploader("Upload an image")

if file:
    img = load_image(file)
    predictions = predict(img, model, device)
    st.title("Here is the image you uploaded")
    resized_image = img.resize((340, 340))
    st.image(resized_image)
    st.title("Prediction:")
    st.write(predictions)



/opt/mamba/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/mamba/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
!streamlit run main.py


  You can now view your Streamlit app in your browser.

  Network URL: http://10.233.127.249:8501
  External URL: http://185.24.184.227:8501

^C
  Stopping...
